#ClMATE cohort analysis - Year 11 15/16 mock results

###Use and description of code

This notebook is designed to take the ClMATE database and generate a list of the wost performing 10 topics for each tier. The rationale of the design is as follows:
1. Pull out *all* results for the current year 11 (these are the only assessments in the database at present so this **will** break once more assessments are stored)
2. Clean up the representation to pull out a topic, mark and available mark for each result.
3. Use this to generate a percentage success *(decimal from 0.0 - 1.0)* for each result.
4. Group results by **question name** *(see note below)* and find the mean for each question topic
5. Sort and slice off the first 10 questions

In [49]:
from collections import namedtuple
import pandas as pd
import numpy as np
import sqlite3

with sqlite3.connect("ClMATE_DB.db") as DB:
    query = "SELECT * FROM results WHERE aID in (?, ?)"
    # -- Format is: aID; UPN; teaching_set; aName; qNum; pMark
    resHigher = DB.execute(query, (56, 57)).fetchall()
    resFound = DB.execute(query, (58, 59)).fetchall()

    # -- prep question mark data for combining into the result tuples
    assessmentMarks = dict()
    for a in [56, 57, 58, 59]:
        query = "SELECT qNum, qMark from assessments where aID = ?"
        qScores = DB.execute(query, (a,)).fetchall()
        for q in qScores:
            key = str(a) + '/' + str(q[0])
            assessmentMarks[key] = q[1]

    titleDict = dict()
    for aID in [56, 57, 58, 59]:
        titleDict[aID] = dict()
        with sqlite3.connect("ClMATE_DB.db") as DB:
            query = "SELECT qNum, qTitle FROM assessments where aID = ?"
            qTitles = DB.execute(query, (aID,)).fetchall()
            for q in qTitles:
                titleDict[aID][q[0]] = q[1].strip()

# -- Improve readability for main dataset: class_results should NOT be mutated
result = namedtuple('Result', ['topic', 'performance'])
higher_results = [result(titleDict[r[0]][r[4]], r[5] / assessmentMarks[str(r[0]) + '/' + str(r[4])]) for r in resHigher]
found_results = [result(titleDict[r[0]][r[4]], r[5] / assessmentMarks[str(r[0]) + '/' + str(r[4])]) for r in resFound]

# Warning!!
## The following code is super hacky and relies on the fact that - in this case - each question has a unique name accross *all* assessments being agregated over.
## It will *not* work correctly if this is not the case!

In [50]:
higher_df = pd.DataFrame(higher_results, columns=higher_results[0]._fields)
# Convert decimal performance score to a percentage
higher_grouped = higher_df.groupby(by='topic').mean()*100
higher_bottom_10 = higher_grouped.sort_values(by='performance')[1:11]
np.round(higher_bottom_10, 2)

,performance
topic,
Volume and rates of change,4.16
Algebraic proof for recurring decimals,4.43
Surds and indices,6.65
Algebraic proof and solving a quadratic,7.71
Changing the subject of a formula,8.38
Transformations of graphs,9.83
Inequalities and regions,14.16
Using upper and lower bounds in calculations - speed,15.46
Solving problems using similar volume,15.61


In [51]:
found_df = pd.DataFrame(found_results, columns=found_results[0]._fields)
# Convert decimal performance score to a percentage
found_grouped = found_df.groupby(by='topic').mean()*100
found_bottom_10 = found_grouped.sort_values(by='performance')[1:11]
np.round(found_bottom_10, 2)

,performance
topic,
"Shaded region - bisect a line, radius from a point, scales",4.04
Area of compound shape; cost of tiles; % descrease,5.45
Volume of cuboids,6.06
Perimeter of rectangle; pythagoras to 1 d.p.,7.58
Ratio & proportion including decimals,18.18
Factorising; simplify with indices,20.45
Cost based on elec meter readings & price per unit,28.03
Form & solve equation,29.09
"Pie charts, angles and the values they respresent",29.29
